In [2]:
import sys
from functools import reduce
import numpy as np
import cirq
from stabilizer_states import StabilizerStates
from stabilizer_toolkit.decompositions import rank2, validate_decompositions
from stabilizer_toolkit.magic_states import enumerate_ccz
from stabilizer_toolkit.helpers.unitary import get_tensored_unitary

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
np.set_printoptions(precision=3, linewidth=sys.maxsize, edgeitems=4, threshold=1024, suppress=True) 

For four qubit $\mathtt{CCZ}$ magic states, we already know we can search within the real stabilizer states, reducing the dataset down from $36,720$ complex-valued four qubit stabilizer states down to only $4,320$ real-valued stabilizer states.

In [5]:
S4 = StabilizerStates(4)
print(len(S4))

36720


In [6]:
S4 = StabilizerStates(4, 'real')
print(len(S4))

4320


If we brute force search over our real stabilizer state dataset, then this requires looking at $9,329,040$ pairs, which can be slow with our standard decomposition method (~7.5 minutes on my laptop). And this is only for a **single**, distinct four-qubit $\mathtt{CCZ}$ magic state.

In [30]:
_, state, _, _ = next(enumerate_ccz(4))
decompositions, coeffs = rank2.search_all_stabilizer_states(state, S4)

2023-04-20 22:13:38,660	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|██████████████████████████████████████████████████████████████████████| 9329040/9329040 [07:20<00:00, 21201.19it/s]


In [31]:
validate_decompositions(state, decompositions, coeffs)

15 decompositions
|ψ〉	= [ 0.25  0.25  0.25  0.25  0.25  0.25  0.25 -0.25  0.25  0.25  0.25 -0.25  0.25  0.25  0.25  0.25]

✅	= [1.] * [0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
	+ [-0.707] * [0.    0.    0.    0.    0.    0.    0.    0.707 0.    0.    0.    0.707 0.    0.    0.    0.   ]

✅	= [1.] * [ 0.25  0.25  0.25 -0.25  0.25  0.25  0.25 -0.25  0.25  0.25  0.25 -0.25  0.25  0.25  0.25 -0.25]
	+ [0.707] * [0.    0.    0.    0.707 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.707]

✅	= [1.] * [ 0.25  0.25  0.25  0.25  0.25 -0.25  0.25 -0.25  0.25 -0.25  0.25 -0.25  0.25  0.25  0.25  0.25]
	+ [0.707] * [0.    0.    0.    0.    0.    0.707 0.    0.    0.    0.707 0.    0.    0.    0.    0.    0.   ]

✅	= [1.] * [ 0.25 -0.25  0.25  0.25  0.25  0.25  0.25 -0.25  0.25  0.25  0.25 -0.25  0.25 -0.25  0.25  0.25]
	+ [0.707] * [0.    0.707 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.707 0.    0.   ]

✅	= [1.]

True

There are $3$ distinct four qubit $\mathtt{CCZ}$ circuits and corresponding magic states, so this would take over 20 minutes to calculate all rank-2 decompositions.

In [22]:
print(len(list(enumerate_ccz(4))))

3


If we use our ternary search method, then this is much faster (~1 second on my laptop). We can go through all distinct four qubit magic states now in a matter of seconds. Each distinct state has $15$ rank-2 decompositions characteristic of the the three qubit $|\mathtt{CCZ}\rangle$ magic state. We know from the manuscript that each four qubit $\mathtt{CCZ}$ magic state reduces down to a single $\mathtt{CCZ}$ in the circuit for generating the magic state.

In [23]:
for index, state, D, circuit in enumerate_ccz(4):
    decompositions, coeffs = rank2.ternary_search(state, S4)
    print()
    print(f"Distinct circuit index {index}")
    print(f"|ψ〉= {state}")
    print(f" D = diag({np.diag(D)})")
    print(circuit)
    valid = validate_decompositions(state, decompositions, coeffs, show=False)
    status = "✅" if valid else "❌"
    print(f"All {len(decompositions)} decomposition(s) rank-2: {status}")
    print()

2023-05-11 22:45:13,629	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|█████████████████████████████████████████████████████████████████████████████| 4320/4320 [00:02<00:00, 1860.13it/s]



Distinct circuit index 3
|ψ〉= [ 0.25  0.25  0.25  0.25  0.25  0.25  0.25 -0.25  0.25  0.25  0.25 -0.25  0.25  0.25  0.25  0.25]
 D = diag([ 1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.  1.])
[[1 0]
 [0 1]
 [1 1]
 [1 1]]
All 15 decomposition(s) rank-2: ✅



2023-05-11 22:45:22,735	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|█████████████████████████████████████████████████████████████████████████████| 4320/4320 [00:02<00:00, 1888.93it/s]



Distinct circuit index 7
|ψ〉= [ 0.25  0.25  0.25  0.25  0.25  0.25  0.25 -0.25  0.25  0.25  0.25 -0.25  0.25 -0.25  0.25 -0.25]
 D = diag([ 1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1. -1.  1. -1.])
[[1 1 0]
 [1 0 1]
 [0 1 1]
 [1 1 1]]
All 15 decomposition(s) rank-2: ✅



2023-05-11 22:45:30,999	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|█████████████████████████████████████████████████████████████████████████████| 4320/4320 [00:02<00:00, 1885.55it/s]



Distinct circuit index 15
|ψ〉= [ 0.25  0.25  0.25  0.25  0.25  0.25  0.25 -0.25  0.25  0.25  0.25 -0.25  0.25 -0.25 -0.25  0.25]
 D = diag([ 1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1. -1. -1.  1.])
[[1 1 1 0]
 [1 1 0 1]
 [1 0 1 1]
 [0 1 1 1]]
All 15 decomposition(s) rank-2: ✅

